Given a string S and a string T, count the number of distinct subsequences of S which equals T.

A subsequence of a string is a new string which is formed from the original string by deleting some (can be none) of the characters without disturbing the relative positions of the remaining characters. (ie, "ACE" is a subsequence of "ABCDE" while "AEC" is not).

After seeing various dynamic programming problems, I sort of get the hang of a type of them. In this problem at each step, we either pick an element of "S" or we don't, 0/1 knapsack problem.

Define `D[i][j]` as the number of ways to write `T[:j+1]` using `S[:i+1]`.

Let's take a look at the first example on LeetCode, `S = "rabbbit"` and `T = "rabbit"`. Specifically let's look at the first row. How many ways can we use the letter `r` to write `r`? 1. What about using the letter `r` to write `ra`? 0. And so on. So that we obtain the following:
```
 rabbit
r100000
```
We can continue and fill out the table to get
```
 rabbit
r100000
a110000
b111000
b112100
b113300
i113330
t113333
```
Looking for patterns, it seems that `D[i][j]` is the sum of `D[i-1][j]` and `D[i-1][j-1]` when `S[i] == T[j]`, otherwise it's just equal to `D[i-1][j]`.

As a matter of interpretation, when we have an extra letter to use to build `T[:j+1]` then we can not use it in which case we've already counted the number of ways to build it (`D[i][j-1]`) or we can try and add it. We are allowed to add it if `S[i] == T[j]` and we are adding the character to `T[:j]` so the number of ways to use `S[:i]` to make `T[:j]` is exactly given by `D[i-1][j-1]`.

In [22]:
class Solution:
    def numDistinct(self, s, t):
        D = [1] + [0]*len(t) # The number of ways to write the empty string with the empty string is 1.
        
        for char_s in s:
            D_new = [1]
            for i, char_t in enumerate(t):
                D_new.append(D[i+1] + (D[i] if char_s == char_t else 0))
            D = D_new
            print(D)
            
        return D[-1]

If we wanted to use even less memory, then we can just store `D[i+1]` and then update `D[i+1]` in place.
```
        for char_s in s:
            prev = D[0]
            for i, char_t in enumerate(t):
                prev, D[i+1] = D[i+1], D[i+1] + 0 if char_s != char_t else prev
            print(D)
```

In terms of runtime, a quick way to avoid unnecessary work is to immediately return 0 if len(s) < len(t).

```
        if len(s) < len(t):
            return 0
```

Next we note that we make a pass through all the letters `char_t` of `t` for every `char_s`. However, we're doing too much work because we only need to make updates to `D` when `char_s == char_t`. So if put initial effort into passing through `t` and index its letters first, then we can immediately retrieve those indices matching `char_s`.

```
        t_dict = {}
        for i, char_t in enumerate(t):
            t_dict.setdefault(char_t, []).append(i)
        
        for char_s in s:
            prev_i = 0
            prev = D[0]
            for i in t_dict.get(char_s, []):
                if i == prev_i: # if i + 1 == prev_i + 1:
                    # `D[i]` was disturbed in previous step so we use the stored value `prev`
                    prev, prev_i, D[i+1] = D[i+1], i+1, D[i+1] + prev
                else: 
                    # `D[i]` was undisturbed in previous step so necessarily we must use it
                    prev, prev_i, D[i+1] = D[i+1], i+1, D[i+1] + D[i]
            print(D)
```

On Leetcode this newer solution has a 32 ms runtime (13.1 MB memory) ("faster than 98.41%") versus the first solution which had a 128 ms runtime (12.9 MB memory).

Note that more succinctly we can write the following:
```
            for i in t_dict.get(char_s, []):
                prev, prev_i, D[i+1] = D[i+1], i+1, D[i+1] + (prev if i == prev_i else D[i])
```

I then realized that the reason for all the bookkeeping is because values are getting replaced. However, we don't have to bookkeep if we go *backwards*!

```
            for i in t_dict.get(char_s, [])[::-1]:
                D[i+1] += D[i]
```

On Leetcode this newer solution has a 28 ms runtime (12.8 MB memory) ("faster than 99.34%").

In [74]:
class Solution:
    def numDistinct(self, s, t):
        if len(s) < len(t):
            return 0
        
        t_dict = {}
        for i, char_t in enumerate(t):
            t_dict.setdefault(char_t, []).append(i)
        
        D = [1] + [0]*len(t)
        
        for char_s in s:
            for i in t_dict.get(char_s, [])[::-1]:
                D[i+1] += D[i]
            print(D)
            
        return D[-1]

Finally I was curious if the following code would help burn through certain test cases.

```
        i = -1
        for char_t in t:
            while i < len(s):
                i += 1
                try:
                    if char_t == s[i]:
                        break
                except:
                    return 0
```

It turns out the answer is yes. On Leetcode this newer solution has a 20 ms runtime (12.9 MB memory) ("faster than 100.00%").             

In [98]:
class Solution:
    def numDistinct(self, s, t):
        # If len(s) is less than len(t), there's no way to write t using s
        if len(s) < len(t):
            return 0
        
        # We check if t can be written at least one way using s
        i = -1
        for char_t in t:
            while i < len(s):
                i += 1
                try:
                    if char_t == s[i]:
                        break
                except:
                    return 0
                
        # Since there's at least one way to write t using s, now we count the ways.
        t_dict = {}
        for i, char_t in enumerate(t):
            t_dict.setdefault(char_t, []).append(i)
        
        D = [1] + [0]*len(t)
        
        for char_s in s:
            for i in t_dict.get(char_s, [])[::-1]:
                D[i+1] += D[i]
            print(D)
            
        return D[-1]

Remark: I suppose it depends on the specific test cases. If one combines the loop for checking at least one way to write t using s and the loop which creates t_dict, then LeetCode ran that code in 24 ms (13 MB memory); still faster than 100%.

# Test Cases

In [99]:
s = Solution()

In [100]:
s.numDistinct("rabbbit", "rabbit")

[1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0]
[1, 1, 1, 2, 1, 0, 0]
[1, 1, 1, 3, 3, 0, 0]
[1, 1, 1, 3, 3, 3, 0]
[1, 1, 1, 3, 3, 3, 3]


3

In [101]:
s.numDistinct("baggbag", "bag")

[1, 1, 0, 0]
[1, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 2]
[1, 2, 1, 2]
[1, 2, 3, 2]
[1, 2, 3, 5]


5

In [102]:
s.numDistinct("baggbag", "")

[1]
[1]
[1]
[1]
[1]
[1]
[1]


1

In [103]:
s.numDistinct("", "")

1

In [104]:
s.numDistinct("", "a")

0

In [105]:
s.numDistinct("aaaaa", "a")

[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]


5

In [106]:
s.numDistinct("aaaaabb", "ab")

[1, 1, 0]
[1, 2, 0]
[1, 3, 0]
[1, 4, 0]
[1, 5, 0]
[1, 5, 5]
[1, 5, 10]


10

In [107]:
s.numDistinct("aacabbacabcbb", "acbb")

[1, 1, 0, 0, 0]
[1, 2, 0, 0, 0]
[1, 2, 2, 0, 0]
[1, 3, 2, 0, 0]
[1, 3, 2, 2, 0]
[1, 3, 2, 4, 2]
[1, 4, 2, 4, 2]
[1, 4, 6, 4, 2]
[1, 5, 6, 4, 2]
[1, 5, 6, 10, 6]
[1, 5, 11, 10, 6]
[1, 5, 11, 21, 16]
[1, 5, 11, 32, 37]


37

In [108]:
s.numDistinct("daadcabbacabcbb", "acbb")

[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[1, 2, 0, 0, 0]
[1, 2, 0, 0, 0]
[1, 2, 2, 0, 0]
[1, 3, 2, 0, 0]
[1, 3, 2, 2, 0]
[1, 3, 2, 4, 2]
[1, 4, 2, 4, 2]
[1, 4, 6, 4, 2]
[1, 5, 6, 4, 2]
[1, 5, 6, 10, 6]
[1, 5, 11, 10, 6]
[1, 5, 11, 21, 16]
[1, 5, 11, 32, 37]


37

In [109]:
s.numDistinct("aacabbacabcbb", "acbbdb")

0